# Buildkite API development

In [ ]:
!cd ..

In [ ]:
import dotenv

In [ ]:
dotenv.load_dotenv()

In [ ]:
import os

token = os.getenv("BUILDKITE_PAT")

In [ ]:
import requests

base_url = "https://api.buildkite.com/v2"

h = {"Authorization": f"Bearer {token}"}

In [ ]:
requests.get(base_url + "/organizations/clima/pipelines", headers=h).json()